<a href="https://colab.research.google.com/github/gustavoalcantara/data-portfolio/blob/main/desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Desafio CIVITAS - EMD**

##**Gustavo C. Alcântara**

###**Analista de Dados**

#####Proposta: O objetivo é fazer uma análise exploratória dos dados, identificar inconsistências, além de identificar placas de veículos que foram possivelmente clonadas, usando as informações disponíveis.



**Introdução e Metodologia**

De acordo com a proposta do desafio e da documentação fornecida na página do [Escritório de Dados do Rio](https://docs.dados.rio/tutoriais/como-acessar-dados/#acessando-dados-via-bigquery), abaixo apresento as bibliotecas necessárias para realizar a análise exploratória dos dados e as inconcistências presentes no `Dataframe` (df).

Para isso, iremos utilizar o pacote `basedosdados` para acesso ao df via bigquery; `geopandas`, `folium` e `pandas` para manipulação, plot e análise espacial das variáveis.
Todas as queries encontram-se comentadas caso haja necessidade de revisão.

In [2]:
import basedosdados as bd
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
import geopandas as gpd
import pandas as pd
import mapclassify
import numpy as np
from scipy import stats

##**Análise Exploratória dos Dados**
###**Overview do Dataframe**
- A tabela `readings_2024_06` presente no conjunto `rj-cetrio.desafio` não encontra-se particionada.
- Existe um total de `36.358.536` de observações durante o período de `6/6/2024` a `13/06/2024`.
- Existe um total de `1.816.325` de valores nulos para a variável `datahora_captura`. Como essa variável indica a data e hora do recebimento dos dados, pode ser que tenha sido um problema nesse processo de recebimento.
- Todas as queries realizadas para a análise exploratória e verificação de placas clonadas você pode acessar [aqui](https://console.cloud.google.com/bigquery?sq=30984071736:74856d5bbbfa439f842cb3df444f646c)

###**Análise Exploratória Descritiva**
- Ao total existem 615 Radares distribuidos na cidade que possuem Latitude e Longitude;
- A empresa que mais possui multas é, por consequência, a empresa que mais possui radares. Foram aplicadas `24.474.488` multas pela empresa `
HiVFr51Ixg==` que detém 408 radares.
- A velocidade Média das Multas encontra-se em `32.8 km/h`
- O Radar que possui mais multas está localizado na Avenida das Américas, sendo um total de `276228` no período analisado.
- Os Principais horários de ocorrência de multas estão no período entre 10h a 13h, sendo um total de `9.395.713` multas aplicadas neste horário.

##**Mapa da Distribuição das Multas**
Por ser geógrafo de formação, gosto muito de apresentar os dados no mapa. Portanto, as queries que apresentarei a seguir trazem consigo uma `análise espacial` que pode contribuir em ações assertivas à Civitas.

###**Distribuição das localidades de Multas na cidade do Rio de Janeiro**

In [9]:
#Quantidade de Multas por Lat/Long.
#Observe que existe uma longitude erroneamente atribuída. Podemos excluir essas observações ou corrigir este valor.
#Optei por corrigir este valor e atribuir o `-` para este ponto em especifico.
query = '''
WITH tabela AS(
SELECT
camera_latitude,
camera_longitude,
COUNT (*) AS total_multas,
AVG(velocidade) AS velocidade_media
FROM `rj-cetrio.desafio.readings_2024_06`
WHERE camera_latitude != 0 AND #Foi verificado que existem lat/long com 0 atribuido
  camera_longitude !=0
GROUP BY camera_latitude, camera_longitude)
SELECT * FROM tabela
ORDER BY total_multas DESC
'''

df = bd.read_sql(query, billing_project_id='proforest-aula-1', reauth = True)
print(df.describe())

Downloading: 100%|██████████|
       camera_latitude  camera_longitude  total_multas  velocidade_media
count       615.000000        615.000000         615.0        615.000000
mean        -22.926978        -43.198863  58662.297561         32.897419
std           0.074022          3.497022  52206.455882         12.113502
min         -23.859722        -43.690229           1.0          0.000000
25%         -22.974836        -43.394819       16184.5         27.823306
50%         -22.920839        -43.331540       47260.0         34.259088
75%         -22.885628        -43.242500       85842.5         39.461045
max         -22.485097         43.334218      276228.0         80.091170


In [10]:
#Correção
df['camera_longitude'] = df['camera_longitude'].apply(lambda x: -abs(x) if x > 0 else x)
print(df.describe())

       camera_latitude  camera_longitude  total_multas  velocidade_media
count       615.000000        615.000000         615.0        615.000000
mean        -22.926978        -43.339788  58662.297561         32.897419
std           0.074022          0.117913  52206.455882         12.113502
min         -23.859722        -43.690229           1.0          0.000000
25%         -22.974836        -43.394819       16184.5         27.823306
50%         -22.920839        -43.331564       47260.0         34.259088
75%         -22.885628        -43.242917       85842.5         39.461045
max         -22.485097        -43.122325      276228.0         80.091170


In [11]:
#Transformar meu df em um geodataframe para plotar no mapa:
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.camera_longitude, df.camera_latitude))

In [12]:
#Quantas quebras possíveis? Fiz considerando quantis.
histograma = mapclassify.Quantiles(df['total_multas'], k=5)
histograma

Quantiles

       Interval          Count
------------------------------
[     1.00,  11474.20] |   123
( 11474.20,  31207.20] |   123
( 31207.20,  61941.80] |   123
( 61941.80,  96601.40] |   123
( 96601.40, 276228.00] |   123

In [46]:
mapa = folium.Map(location=[-22.9, -43.2], zoom_start=12)
for index, row in df.iterrows():
    lat, lon = row['camera_latitude'], row['camera_longitude']
    total_multas = row['total_multas']

    if total_multas > 96601: #Atribui essa quebra com base ns quebras naturais por Jenks
        color = 'red'
    elif total_multas > 61941:
        color = 'orange'
    elif total_multas > 31207:
        color = 'yellow'
    elif total_multas > 11474:
        color = 'green'
    else:
        color = 'blue'

# Optei por fazer um Mapa coroplético por bolhas. Por favor, clique em cada bollha
#para aparecer a quantidade de Multas que cada radar possui.
    folium.CircleMarker(
        location=[lat, lon],
        radius=total_multas / 10000,  # Ajuste o tamanho conforme necessário
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f'Total de Multas: {total_multas}'
    ).add_to(mapa)

# Salvar o mapa como um arquivo HTML
mapa.save('mapa_multas.html')

mapa

#**Análise de Placas Duplicadas**
Em uma análise da contagem de multas por placa, foi possível observar que:
- Existe um total de `7.984.610` placas diferentes.
- 75% das placas possuem até 3 multas no período analisado.
- As placas que foram clonadas concentram-se nos radares na Avenida das Américas e Avenida Brasil.
- Os principais horários que as placas clonadas passaram pelos radares foi das 13h às 17h.

###**Mapa dos Radares que mais possuem placas clonadas**

In [14]:
query = '''
SELECT
    camera_latitude,
    camera_longitude,
    COUNT(DISTINCT placa) AS placas_clonadas
FROM
    `rj-cetrio.desafio.readings_2024_06`
WHERE camera_latitude !=0 AND camera_longitude !=0
GROUP BY
    camera_latitude, camera_longitude
HAVING
    COUNT(DISTINCT placa) > 10 --Considerei acima de 10 multas para determinar o que foi clonado.
ORDER BY
    placas_clonadas DESC;
'''

df = bd.read_sql(query, billing_project_id='proforest-aula-1', reauth = True)
print(df.describe())

Downloading: 100%|██████████|
       camera_latitude  camera_longitude  placas_clonadas
count       613.000000        613.000000            613.0
mean        -22.927127        -43.198777      42207.33279
std           0.074081          3.502731     38707.829492
min         -23.859722        -43.690229             12.0
25%         -22.975860        -43.395011          12744.0
50%         -22.920913        -43.331564          32484.0
75%         -22.885694        -43.242500          59486.0
max         -22.485097         43.334218         202144.0


In [15]:
#Atribuindo como um geodataframe
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.camera_longitude, df.camera_latitude))

#Corrigind a longitude
df['camera_longitude'] = df['camera_longitude'].apply(lambda x: -abs(x) if x > 0 else x)
print(df.describe())

       camera_latitude  camera_longitude  placas_clonadas
count       613.000000        613.000000            613.0
mean        -22.927127        -43.340161      42207.33279
std           0.074081          0.117913     38707.829492
min         -23.859722        -43.690229             12.0
25%         -22.975860        -43.395011          12744.0
50%         -22.920913        -43.332325          32484.0
75%         -22.885694        -43.243333          59486.0
max         -22.485097        -43.122325         202144.0


In [16]:
#Verificar a quantidade de Quantis possíveis
histograma = mapclassify.Quantiles(df['placas_clonadas'], k=5)
histograma

Quantiles

       Interval          Count
------------------------------
[    12.00,   9417.60] |   123
(  9417.60,  23429.00] |   122
( 23429.00,  43256.20] |   123
( 43256.20,  67361.00] |   122
( 67361.00, 202144.00] |   123

In [18]:
mapa = folium.Map(location=[-22.9, -43.2], zoom_start=12)
for index, row in df.iterrows():
    lat, lon = row['camera_latitude'], row['camera_longitude']
    placas_clonadas = row['placas_clonadas']

    if placas_clonadas > 67361: #Atribui essa quebra com base ns quebras naturais por Jenks
        color = 'red'
    elif placas_clonadas > 43256 :
        color = 'orange'
    elif placas_clonadas > 23429 :
        color = 'yellow'
    elif placas_clonadas > 9417:
        color = 'green'
    else:
        color = 'blue'

# Optei por fazer um Mapa coroplético por bolhas. Por favor, clique em cada bollha
#para aparecer a quantidade de Multas que cada radar possui.
    folium.CircleMarker(
        location=[lat, lon],
        radius=placas_clonadas / 10000,  # Ajuste o tamanho conforme necessário
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f'Placas Clonadas: {placas_clonadas}'
    ).add_to(mapa)

# Salvar o mapa como um arquivo HTML
mapa.save('mapa_multas.html')

mapa

#**Considerações**
O presente desafio é uma proposta e apresentação de uma análise exploratória dos dados de radar da cidade do Rio de Janeiro.

Para a problemática das placas duplicadas, uma das soluções passíveis seria fazer blitz nos principais horários próximos aos radares que mais possuem placas duplicadas para se coibir este tipo de atividade.

Para a problemática da velocidade, pode ser interessante instalar lombadas ou investir em conscientização das pessoas no trânisto. É possível verificar a presença 131 escolas em um raio de 100 metros de cada radar que mais possuem multas por excesso de velocidade.

Portanto, cabe focar em ações de conscientização e educação no trânsito, além das intervenções legais para a redução da quantidade de placas clonadas circulando na cidade.